<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Используемые-библиотеки" data-toc-modified-id="Используемые-библиотеки-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Используемые библиотеки</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Анализ-и-обработка-данных" data-toc-modified-id="Анализ-и-обработка-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ и обработка данных</a></span></li><li><span><a href="#Предобучение" data-toc-modified-id="Предобучение-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Предобучение</a></span></li><li><span><a href="#Токенизация" data-toc-modified-id="Токенизация-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Токенизация</a></span></li><li><span><a href="#Эмбеддинг" data-toc-modified-id="Эмбеддинг-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Эмбеддинг</a></span></li><li><span><a href="#Разделение-данных-на-выборки" data-toc-modified-id="Разделение-данных-на-выборки-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Разделение данных на выборки</a></span></li><li><span><a href="#Выбор-и-обучение-модели" data-toc-modified-id="Выбор-и-обучение-модели-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Выбор и обучение модели</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Заключение" data-toc-modified-id="Заключение-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Заключение</a></span></li></ul></div>

# Классификация на эмбеддингах

# Анализ текстов

**Цель и задача проекта**

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Требуется инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.
Модель со значением метрики качества F1 не меньше 0.75. 

## Используемые библиотеки

In [247]:
import numpy as np
import pandas as pd
import torch # обработки естественного текста и компьютерного зрения.
import transformers
from tqdm import notebook # подключим библиотеку tqdm (араб. taqadum, تقدّم, «прогресс»). 
# Она нужна, чтобы наглядно показать индикатор прогресса. В Jupyter применим функцию notebook() из этой библиотеки
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertModel, BertTokenizer

## Загрузка данных

In [265]:
df_tweets = pd.read_csv('tweets.csv')
df_tweets = df_tweets.sample(400).reset_index(drop=True)


## Анализ и обработка данных

In [249]:
df_tweets['positive']

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    0
397    0
398    1
399    1
Name: positive, Length: 400, dtype: int64

In [250]:
df_tweets

,text,positive
0,Мой авторитет среди одногруппников на нуле:( ...,0
1,Впервые за долгое время матч буду смотреть дома(,0
2,Обо мне...((( “@trollgnet: Сначала ты отдаёшьс...,0
3,"такое настроение странное- ничего не хочу, ник...",0
4,Раздайте кто нить пожалуйста скорость ну о ч м...,0
...,...,...
395,Идёшь на концерт известного артиста? Не забыва...,1
396,@Len_o_k он не идет на контакт ( ахаха http://...,0
397,Пятеро жителей Чечни задержаны за драку у кино...,0
398,Опааа)) Нашлось фото с Глендейла ахахах http:/...,1


In [251]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      400 non-null    object
 1   positive  400 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.4+ KB


Пропусков нет. Тип признака positive изменим с целью экономии памяти.

In [252]:
df_tweets['positive'] = df_tweets['positive'].astype('bool')
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      400 non-null    object
 1   positive  400 non-null    bool  
dtypes: bool(1), object(1)
memory usage: 3.6+ KB


## Предобучение

In [253]:
model_name = 'DeepPavlov/rubert-base-cased'

model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Токенизация

Инициализируем токенизатор как объект класса BertTokenizer().

In [254]:
tokenizer = BertTokenizer.from_pretrained(model_name)

Преобразуем текст в номера токенов из словаря методом encode(). Для корректной работы модели мы указали аргумент add_special_tokens (англ. «добавить специальные токены»), равный True. Это значит, что к любому преобразуемому тексту добавляется токен начала (101) и токен конца текста (102). 

In [255]:
tokenized = df_tweets['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

Применим метод padding (англ. «отступ»), чтобы после токенизации длины исходных текстов в корпусе были равными. Только при таком условии будет работать модель BERT. Пусть стандартной длиной вектора n будет длина наибольшего во всём датасете вектора. Остальные векторы дополним нулями:

In [256]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

Теперь поясним модели, что нули не несут значимой информации. Это нужно для компоненты модели, которая называется «внимание» (англ. attention). Отбросим эти токены и «создадим маску» для действительно важных токенов, то есть укажем нулевые и не нулевые значения:

In [257]:
attention_mask = np.where(padded != 0, 1, 0)

## Эмбеддинг

Эмбеддинги модель BERT создаёт батчами. Чтобы хватило оперативной памяти, сделаем размер батча = 100.

In [258]:
batch_size = 100

Сделаем цикл по батчам. Отображать прогресс будет функция notebook()
Преобразуем данные в формат тензоров (англ. tensor) — многомерных векторов в библиотеке torch. Тип данных LongTensor (англ. «длинный тензор») хранит числа в «длинном формате», то есть выделяет на каждое число 64 бита.

In [259]:
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        # Для ускорения вычисления функцией no_grad() (англ. no gradient, «нет градиента») в библиотеке torch 
        # укажем, что градиенты не нужны: модель BERT обучать не будем.
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)# Чтобы получить эмбеддинги 
            # для батча, передадим модели данные и маску
        # Из полученного тензора извлечём нужные элементы и добавим в список всех эмбеддингов:
        embeddings.append(batch_embeddings[0][:,0,:].numpy())#преобразуем элементы методом numpy()к типу numpy.array

  0%|          | 0/4 [00:00<?, ?it/s]

In [260]:
# Соберём все эмбеддинги в матрицу признаков вызовом функции concatenate():
features = np.concatenate(embeddings)

## Разделение данных на выборки

In [261]:
features_train, features_test, target_train, target_test = train_test_split(
    features, df_tweets['positive'], test_size=0.5, random_state=12345)

In [262]:
print(features_train)
features_train.shape

[[ 0.13274057 -0.04342028  0.56878644 ...  0.34261847  0.52688426
  -0.13554862]
 [-0.12058353 -0.14904153 -0.24008556 ...  0.25007153  0.35169622
  -0.24301112]
 [ 0.07326889 -0.21501686  0.43821698 ...  1.1929369   0.9278865
   0.00945342]
 ...
 [ 0.02973904 -0.2922577  -0.0863848  ...  0.75324607  0.4777244
  -0.08358181]
 [-0.03922073 -0.46211317  0.82884085 ...  0.81092113  0.57842416
  -0.23604804]
 [ 0.10130152  0.00850299 -0.05558029 ...  0.10352381 -0.01167706
  -0.15645406]]


(200, 768)

## Выбор и обучение модели

In [263]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
model.fit(features_train, target_train) 

LogisticRegression(max_iter=1000, random_state=12345)

## Тестирование модели

In [264]:
predictions = model.predict(features_test)
accuracy_score(target_test, predictions)

0.945

## Заключение

Мы провели работу по созданию инструмента для определения токсичных комментариев для интернет-магазина. Был проделан следующий объём работ:

+ Данные:
  - загружены, 
  - исследованы и
  - оптимизированы.
    
+ Провели предобучение - модель `DeepPavlov/rubert-base-cased`,
+ Выполнили токенизацию инструментом `BertTokenizer()`,
+ И эмбеддинг.


Полученная модель показала существенно лучший результат f1, чем заданное заказчиком 0,75.
Для производства выбрана логистическая регрессия.

